<a href="https://colab.research.google.com/github/ujjawal-sharma-2005/unit1/blob/main/Optical_Flow_and_Motion_Tracking_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# UNIT 4: Optical Flow and Motion Tracking Demo (SAFE VERSION)
# ============================================================

import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video
from google.colab import files
import imageio
import tempfile

print("🎥 Upload a short MP4/AVI video (optional).")
print("If you skip, a synthetic motion video will be created.")

uploaded = files.upload()

# ------------------------------------------------------------
# 1️⃣ Handle video source (upload or synthetic)
# ------------------------------------------------------------
if len(uploaded) == 0:
    print("⚙️ No video uploaded. Creating synthetic motion video...")

    h, w = 256, 256
    out = cv2.VideoWriter('synthetic_motion.avi',
                          cv2.VideoWriter_fourcc(*'XVID'),
                          15, (w, h))
    for i in range(60):
        frame = np.zeros((h, w, 3), dtype=np.uint8)
        cv2.circle(frame, (30 + i*3, 128), 20, (255, 255, 255), -1)
        out.write(frame)
    out.release()
    video_path = "synthetic_motion.avi"
    print("✅ Synthetic video created.")
else:
    video_path = list(uploaded.keys())[0]
    print(f"✅ Using uploaded video: {video_path}")

# ------------------------------------------------------------
# 2️⃣ Initialize capture and resize first frame
# ------------------------------------------------------------
cap = cv2.VideoCapture(video_path)
ret, first_frame = cap.read()
if not ret:
    raise ValueError("❌ Could not read video. Try another file or format.")

# Normalize size
H, W = first_frame.shape[:2]
if max(H, W) > 512:
    scale = 512 / max(H, W)
    first_frame = cv2.resize(first_frame, (int(W * scale), int(H * scale)))
H, W = first_frame.shape[:2]

first_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
p0 = cv2.goodFeaturesToTrack(first_gray, mask=None, **feature_params)

lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
color = np.random.randint(0, 255, (100, 3))
mask = np.zeros_like(first_frame)
frames = []

# ------------------------------------------------------------
# 3️⃣ Safe Optical Flow Loop
# ------------------------------------------------------------
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("✅ End of video or read complete.")
        break

    frame_idx += 1

    # Ensure same size for all frames
    if (frame.shape[0] != H) or (frame.shape[1] != W):
        frame = cv2.resize(frame, (W, H))

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    try:
        p1, st, err = cv2.calcOpticalFlowPyrLK(first_gray, gray, p0, None, **lk_params)
    except cv2.error as e:
        print(f"⚠️ Frame {frame_idx} skipped due to size/flow issue: {e}")
        continue

    if p1 is None or st is None:
        print("⚠️ No valid motion vectors found. Ending tracking.")
        break

    good_new = p1[st == 1]
    good_old = p0[st == 1]

    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 3, color[i].tolist(), -1)

    output = cv2.add(frame, mask)
    frames.append(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))

    first_gray = gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()

# ------------------------------------------------------------
# 4️⃣ Display the result
# ------------------------------------------------------------
if len(frames) > 0:
    temp_path = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False).name
    imageio.mimsave(temp_path, frames, fps=15)
    print(f"✅ Motion tracking complete. Frames processed: {len(frames)}")
    display(Video(temp_path, embed=True))
else:
    print("❌ No frames processed — video might be incompatible.")


🎥 Upload a short MP4/AVI video (optional).
If you skip, a synthetic motion video will be created.


Saving 0_Volleyball_Ball_3840x2160.mp4 to 0_Volleyball_Ball_3840x2160.mp4
✅ Using uploaded video: 0_Volleyball_Ball_3840x2160.mp4
⚠️ No valid motion vectors found. Ending tracking.
✅ Motion tracking complete. Frames processed: 91
